# Think Bayes

Second Edition

Copyright 2020 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py and create directories

import os

if not os.path.exists('utils.py'):
    !wget https://github.com/AllenDowney/ThinkBayes2/raw/master/code/soln/utils.py
        
if not os.path.exists('figs'):
    !mkdir figs
        
if not os.path.exists('tables'):
    !mkdir tables

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from empiricaldist import Pmf
from utils import decorate, savefig, write_table

## Introduction

This chapter introduces joint distributions, which are an essential tool for working with distributions of more than one variable.

We'll use them to solve a silly problem on our way to solving a real problem.  The silly problem is figuring out how tall two people are, given only that one is taller than the other.  The real problem is rating chess players (or participants in other kinds of competition) based on the outcome of a game.

To construct joint distributions and compute likelihoods for these problems, we will use outer products and similar operations.  And that's where we'll start.

## Outer operations

Many useful operations can be expressed as the "outer product" of two sequences, or another kind of "outer" operation.
Suppose you have sequences like `x` and `y`:

In [4]:
x = [1,3,5]

In [5]:
y = [2,4]

The outer product of these sequences is an array that contains the product of every pair of values, one from each sequence.
There are several ways to compute outer products, but the one I think is the most versatile is a "mesh grid".

NumPy provides a function called `meshgrid` that computes a mesh grid.  If we give it two sequences, it returns two arrays.

In [6]:
X, Y = np.meshgrid(x, y)

The first array contains copies of `x` arranged in rows, where the number of rows is the length of `y`.

In [7]:
X

The second array contains copies of `y` arranged in columns, where the number of columns is the length of `x`.

In [8]:
Y

Because the two arrays are the same size, we can use them as operands for arithmetic functions like multiplication.

In [9]:
X * Y

This is result is the outer product of `x` and `y`.
We can see that more clearly if we put it in a DataFrame:

In [10]:
df = pd.DataFrame(X * Y, columns=x, index=y)
write_table(df, 'table09-01', bold_rows=True)
df

The values from `x` appear as column names; the values from `y` appear as row labels.  

Each element is the product of a value from `x` and a value from `y`.

The outer sum is similar, except that each element is the *sum* of a value from `x` and a value from `y`.

In [11]:
X + Y

We can use mesh grids to compute the "outer" version of any operator or function.
Here's an example with a comparison operator:

In [12]:
X > Y

The result is an array of Boolean values.

It might not be obvious yet why these operations are useful, but we'll see examples soon.

With that, we are ready to take on a new Bayesian problem.

## How tall is A?

Suppose I choose two people from the population of adult males in the U.S.; I'll call them A and B.  If we see that A taller than B, how tall is A?

To answer this question:

1. I'll use background information about the height of men in the U.S. to form a prior distribution of height,

2. I'll construct a joint distribution of height for A and B (and I'll explain what that is);

3. Then I'll update the prior with the information that A is taller, and 

4. From the posterior joint distribution I'll extract the posterior distribution of height for A.

In the U.S. the average height of male adults in 178 cm and the standard deviation is 7.7 cm.  The distribution is not exactly normal, because nothing in the real world is, but the normal distribution is a pretty good model of the actual distribution, so we can use it as a prior distribution for A and B.

Here's an array of equally-spaced values from roughly 3 standard deviations below the mean to 3 standard deviations above.

In [13]:
mean = 178
std = 7.7

qs = np.arange(mean-24, mean+24, 0.5)

SciPy provides a function called `norm` that represents a normal distribution with a given mean and standard deviation, and provides `pdf`, which evaluates the probability distribution function (PDF), which we will use as the prior probabilities.

In [14]:
from scipy.stats import norm

ps = norm(mean, std).pdf(qs)

I'll store the `ps` and `qs` in a `Pmf` that represents the prior distribution, and normalize it.

In [15]:
prior = Pmf(ps, qs)
prior.normalize()

Here's what it looks like.

In [16]:
prior.plot(color='gray')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Approximate distribution of height for men in U.S.')

This distribution represents what we believe about the heights of `A` and `B` before we take into account the data that `A` is taller.

## Joint distribution

The next step is to construct a distribution that represents the probability of every pair of heights, which is called a joint distribution. The elements of the joint distribution are

$P(A_x~\mathrm{and}~B_y)$

which is the probability that `A` is $x$ cm tall and `B` is $y$ cm tall, for all values of $x$ and $y$.

At this point all we know about `A` and `B` is that they are male residents of the U.S., so their heights are independent; that is, knowing the height of `A` provides no additional information about the height of `B`.

In that case, we can compute the joint probabilities like this:

$P(A_x~\mathrm{and}~B_y) = P(A_x)~P(B_y)$

Each joint probability is the product of one element from the distribution of `x` and one element from the distribution of `y`.

So if we have `Pmf` objects that represent the distribution of height for `A` and `B`, we can compute the joint distribution by computing the outer product of the probabilities in each `Pmf`.

The following function takes two `Pmf` objects and returns a `DataFrame` that represents the joint distribution.

In [17]:
def make_joint(pmf1, pmf2):
    """Compute the outer product of two Pmfs.
    
    First Pmf goes across the columns;
    second goes down the rows.
    
    pmf1, pmf2: Pmf objects
    
    return: DataFrame
    """
    X, Y = np.meshgrid(pmf1, pmf2)
    return pd.DataFrame(X * Y, columns=pmf1.qs, index=pmf2.qs)

The column names in the result are the quantities from `pmf1`; the row labels are the quantities from `pmf2`.

In this example, the prior distributions for `A` and `B` are the same, so we can compute the joint prior distribution like this:

In [18]:
joint = make_joint(prior, prior)
joint.shape

The result is a `DataFrame` with possible heights of `A` along the columns, heights of `B` along the rows, and the joint probabilities as elements.

If the prior is normalized, the joint prior should also be normalized.

In [19]:
joint.to_numpy().sum()

Notice that we have to convert the `DataFrame` to a `NumPy` array before calling `sum`.  Otherwise, `DataFrame.sum` would compute the sums of the columns and return a `Series`.

In [20]:
joint.sum()

## Visualizing the joint distribution

The following function uses `pcolormesh` to plot the joint distribution.

In [21]:
def plot_joint(joint):
    """Plot a joint distribution.
    
    joint: DataFrame representing a joint PMF
    """
    plt.pcolormesh(joint.columns, joint.index, joint, cmap='Blues')
    plt.colorbar()
    decorate(xlabel='A height in cm',
             ylabel='B height in cm')

Here's what the joint prior distribution looks like.

In [22]:
plot_joint(joint)
decorate(title='Joint prior distribution of height for A and B')
savefig('fig09-01')

As you might expect, the probability is highest (darkest) near the mean and drops off farther from the mean.

Another way to visualize the joint distribution is a contour plot.

In [23]:
def plot_contour(joint):
    """Plot a joint distribution.
    
    joint: DataFrame representing a joint PMF
    """
    plt.contour(joint.columns, joint.index, joint)
    decorate(xlabel='A height in cm',
             ylabel='B height in cm')

In [24]:
plot_contour(joint)
decorate(title='Joint prior distribution of height for A and B')

Each line represents a level of equal probability.  

## Likelihood

Now that we have a joint prior distribution, we can update it with the data, which is that `A` is taller than `B`.

Each element in the joint distribution represents a hypothesis about the heights of `A` and `B`.

To compute the likelihood of every pair of quantities, we can extract the column names and row labels from the prior, like this:

In [25]:
x = joint.columns
y = joint.index

And use them to compute a mesh grid.

In [26]:
X, Y = np.meshgrid(x, y)

`X` contains copies of the values in `x`, which are possible heights for `A`.   `Y` contains copies of the values in `y`, which are possible heights for `B`.

If we compare `X` and `Y`, the result is a Boolean array:

In [27]:
A_taller = (X > Y)
type(A_taller)

In [28]:
A_taller.dtype

To compute likelihoods, I'll use `np.where` to make an array with `1` where `A_taller` is `True` and 0 elsewhere.

In [29]:
a = np.where(A_taller, 1, 0)

To visualize this array of likelihoods, I'll put in a `DataFrame` with the values of `x` as column names and the values of `y` as row labels.

In [30]:
likelihood = pd.DataFrame(a, index=x, columns=y)

Here's what it looks like:

In [31]:
plot_joint(likelihood)
decorate(title='Likelihood of A>B')
savefig('fig09-02')

The likelihood of the data is 1 where `X > Y` and 0 elsewhere.

## The update

We have a prior, we have a likelihood, and we are ready for the update.  As usual, the unnormalized posterior is the product of the prior and the likelihood.

In [32]:
posterior = joint * likelihood

I'll use the following function to normalize the posterior:

In [33]:
def normalize(joint):
    """Normalize a joint distribution.
    
    joint: DataFrame
    """
    prob_data = joint.to_numpy().sum()
    joint /= prob_data
    return prob_data

In [34]:
normalize(posterior)

Here's what it looks like.

In [35]:
plot_joint(posterior)
decorate(title='Joint posterior distribution of height for A and B')
savefig('fig09-03')

All pairs where `B` is taller than `A` have been eliminated.  The rest of the posterior looks the same as the prior, except that it has been renormalized.

## Marginal distributions

The joint posterior distribution represents what we believe about the heights of `A` and `B` given the prior distributions and the information that `A` is taller.

From this joint distribution, we can compute the posterior distributions for `A` and `B`.  To see how, let's start with a simpler problem.

Suppose we want to know the probability that `A` is 180 cm tall.  We can select the column from the joint distribution where `x=180`.

In [36]:
column = posterior[180]

This column contains posterior probabilities for all cases where `x=180`; if we add them up, we get the total probability that `A` is 180 cm tall.

In [37]:
column.sum()

It's about 3%.

Now, to get the posterior distribution of height for `A`, we can add up all of the columns, like this:

In [38]:
column_sums = posterior.sum(axis=0)
column_sums

The argument `axis=0` means we want to add up the columns.

The result is a `Series` that contains every possible height for `A` and its probability.  In other words, it is the distribution of heights for `A`.

We can put it in a `Pmf` like this:

In [39]:
marginal_A = Pmf(column_sums)

When we extract the distribution of a single variable from a joint distribution, the result is called a **marginal distribution**.
The name comes from a common visualization that shows the joint distribution in the middle and the marginal distributions in the margins.

Here's what the marginal distribution for `A` looks like:

In [40]:
marginal_A.plot(label='Posterior for A')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Posterior distribution for A')

Similarly, we can get the posterior distribution of height for `B` by adding up the rows and putting the result in a `Pmf`.

In [41]:
row_sums = posterior.sum(axis=1)
marginal_B = Pmf(row_sums)

Here's what it looks like.

In [42]:
marginal_B.plot(label='Posterior for B', color='C1')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Posterior distribution for B')

The following function takes a joint distribution and an axis number, and returns a marginal distribution.

In [43]:
def marginal(joint, axis):
    """Compute a marginal distribution.
    
    axis=0 returns the marginal distribution of the first variable
    axis=1 returns the marginal distribution of the second variable
    
    joint: DataFrame representing a joint distribution
    axis: int axis to sum along
    
    returns: Pmf
    """
    return Pmf(joint.sum(axis=axis))

In [44]:
marginal_A = marginal(posterior, axis=0)
marginal_B = marginal(posterior, axis=1)

Here's what they look like.

In [45]:
prior.plot(label='Prior', color='gray')
marginal_A.plot(label='Posterior for A')
marginal_B.plot(label='Posterior for B')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Prior and posterior distributions for A and B')
savefig('fig09-04')

As you might expect, the posterior distribution for `A` is shifted to the right and the posterior distribution for `B` is shifted to the left.

We can summarize the results by computing the posterior means:

In [46]:
prior.mean()

In [47]:
marginal_A.mean(), marginal_B.mean()

Based on the observation that `A` is taller than `B`, we are inclined to believe that `A` is a little taller than average, and `B` is a little shorter.

Notice that the posterior distributions are a little narrower than the prior.  We can quantify that by computing their standard deviations.

In [48]:
prior.std()

In [49]:
marginal_A.std(), marginal_B.std()

The standard deviations of the posterior distributions are a little smaller, which means we are a little more certain about the heights of `A` and `B` after we compare them.

## Conditional posteriors

Now suppose we measure `A` and find that he is 170 cm tall.  What does that tell us about `B`?

In the joint distribution, each column corresponds a possible height for `A`.  We can select the column that corresponds to height 170 cm like this:

In [50]:
column_170 = posterior[170]

The result is a `Series` that represents possible heights for `B` and their relative likelihoods.
These likelihoods are not normalized, but we can normalize them like this:

In [51]:
cond_B = Pmf(column_170)
cond_B.normalize()

Making a `Pmf` copies the data by default, so we can normalize `cond_B` without affecting `column_170` or `posterior`.

The result is the conditional distribution of height for `B` given that `A` is 170 cm tall.

Here's what it looks like:

In [52]:
prior.plot(label='Prior', color='gray')
marginal_B.plot(label='Posterior for B', color='C1')
cond_B.plot(label='Condition posterior for B', color='C3')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Prior, posterior and conditional distribution for B')
savefig('fig09-05')

The posterior conditional distribution is cut off at 180 cm, because we have established that `B` is shorter than `A` and `A` is 180 cm.

The mean of the conditional distribution is lower than the mean of the posterior marginal distribution.

In [53]:
marginal_B.mean(), cond_B.mean()

## Dependence and independence

When we constructed the joint prior distribution, I said that the heights of `A` and `B` were independent, which means that knowing one of them provides no information about the other.
In other words, the conditional probability $P(A_x | B_y)$ is the same as the unconditioned probability $P(A_x)$.

But remember, that's only true if $A$ and $B$ are independent.  In the posterior distribution, they are not.  We know that `A` is taller than `B`, so if we know how tall `A` is, that gives us information about `B`.

The conditional distribution we just computed demonstrates this dependence.

## Summary

In this notebook I started with the "outer" operations, like outer product, which we used to construct a joint distribution.

In general, you cannot construct a joint distribution from two marginal distributions, but in the special case where the distributions are independent, you can.

We extended the Bayesian update process we've seen in previous notebooks and applied it to a joint distribution.  Then from the posterior joint distribution we extracted posterior marginal distributions and posterior conditional distributions.

As an exercise, you'll have a chance to apply the same process to problem that's a little more difficult and a lot more useful, updating a chess player's rating based on the outcome of a game.

## Exercises

**Exercise:** Based on the results of the previous example, compute the posterior conditional distribution for `A` given that `B` is 180 cm.

Hint: Use `loc` to select a row from a `DataFrame`.

In [54]:
# Solution goes here

In [55]:
# Solution goes here

**Exercise:** Suppose we have established that `A` is taller than `B`, but we don't know how tall `B` is.
Now we choose a random woman, `C`, and find that she is shorter than `A` by at least 15 cm.  Compute posterior distributions for the heights of `A` and `C`.

The average height for women in the U.S. is 163 cm; the standard deviation is 7.3 cm.

In [56]:
# Solution goes here

In [57]:
# Solution goes here

In [58]:
# Solution goes here

In [59]:
# Solution goes here

In [60]:
# Solution goes here

In [61]:
# Solution goes here

In [62]:
# Solution goes here

In [63]:
# Solution goes here

**Exercise:** [The Elo rating system](https://en.wikipedia.org/wiki/Elo_rating_system) is a way to quantify the skill level of players for games like chess.

It is based on a model of the relationship between the ratings of players and the outcome of a game.  Specifically, if $R_A$ is the rating of player `A` and $R_B$ is the rating of player `B`, the probability that `A` beats `B` is given by the [logistic function](https://en.wikipedia.org/wiki/Logistic_function):

$P(\mathrm{A~beats~B}) = 1 / (1 + 10^{(R_B-R_A)/400})$

The parameters $10$ and $400$ are arbitrary choices that determine the range of the ratings.  In chess, the range is from 100 to 2800.

Notice that the probability of winning depends only on the difference in rankings.  As an example, if $R_A$ exceeds $R_B$ by 100 points, the probability that `A` wins is

In [64]:
1 / (1 + 10**(-100/400))

Suppose `A` has a current rating of 1600, but we are not sure it is accurate.  We could describe their true rating with a normal distribution with mean 1600 and standard deviation 100, to indicate our uncertainty.

And suppose `B` has a current rating of 1800, with the same level of uncertainty.

Then `A` and `B` play and `A` wins.  How should we update their ratings?

To answer this question:

1. Construct prior distributions for `A` and `B`.

2. Use them to construct a joint distribution, assuming that the prior distributions are independent.

3. Use the logistic function above to compute the likelihood of the outcome under each joint hypothesis.  

4. Use the joint prior and likelihood to compute the joint posterior. 

5. Extract and plot the marginal posteriors for `A` and `B`.

6. Compute the posterior means for `A` and `B`.  How much should their ratings change based on this outcome?

In [65]:
# Solution goes here

In [66]:
# Solution goes here

In [67]:
# Solution goes here

In [68]:
# Solution goes here

In [69]:
# Solution goes here

In [70]:
# Solution goes here

In [71]:
# Solution goes here

In [72]:
# Solution goes here

In [73]:
# Solution goes here

In [74]:
# Solution goes here

In [75]:
# Solution goes here

In [76]:
# Solution goes here